##Hello!!
This is the actual code for the full hybrid model consisting of everything that I have stated, pls note that running this will actually take some time
(also making this took much more time than i hoped)

In [ ]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from scipy.signal import find_peaks, savgol_filter
from scipy import stats
from scipy.ndimage import gaussian_filter1d
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.feature_selection import SelectKBest, f_classif
import pickle
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)


In [ ]:
#ablation study flgs for experimenting such that you can test out the differences yourself;
USE_HUBER = True       #Using Huber regression vs ols
USE_MULTIINDEX = True  #Use multi index validation vs NDVI only
USE_SAVGOL = True      #Use savitzky golay vs gausian smoothing
USE_CALIBRATION = True #probabiltiies for reliability

print("hybrid ai forest chnage detction system")

print("generated", pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f"huber Regression: {' yes' if USE_HUBER else 'no (using OLS)'}")
print(f"multi index validation: {'yes' if USE_MULTIINDEX else ' no (using ndvi only)'}")
print(f"savitzkygolay smoothing: {' yes' if USE_SAVGOL else 'no (using Gaussian)'}")
print(f"probability calibration: {' yes' if USE_CALIBRATION else ' no'}\n")

hybrid ai forest chnage detction system
generated 2025-09-19 09:22:55
huber Regression:  yes
multi index validation: yes
savitzkygolay smoothing:  yes
probability calibration:  yes



In [ ]:
#fun fact...this took wayyyy too long
class ForestChangeDetectionSystem:
    def __init__(self, study_area="Amazon Tropical Forest", patch_size=64):
        self.study_area = study_area
        self.patch_size = patch_size
        self.change_classes = [
            'Stable Forest', 'Minor Degradation', 'Moderate Deforestation',
            'Major Deforestation', 'Regrowth/Recovery'
        ]
        self.min_forest_ndvi = 0.4 #min ndvi for what counts as forest
        self.forest_change_thresholds = { #these are the values for figuring out what kind of change happened
            'fire_scar': 0.6,
            'major_deforestation': 0.4,
            'moderate_deforestation': 0.25,
            'minor_degradation': 0.1,
            'regrowth': -0.15,
            'seasonal_variation': 0.05 #small changes probably just seasons
        }
        self.enhanced_time_series_features = None
        self.cnn_spatial_features = None
        self.ancillary_features = None
        self.gbm_model = None #holds the trained gradient boosting model later
        self.feature_selector = None #for picking the best features
        self.label_encoder = LabelEncoder() #convert class names to numbers
        self.feature_scaler = RobustScaler() #scale features so model works better
        self.model_metrics = {} #stores how well the model did

    def simulate_satellite_data(self, n_pixels=7500, n_timesteps=48):
        print("generating satellite time series data")

        time_series_ndvi = []
        time_series_nbr = []
        time_series_ndmi = []
        time_series_evi = []
        labels = []
        pixel_coordinates = []
        change_metadata = []
        spatial_tiles = [] #gonna use this for spatial split later
        for pixel_id in range(n_pixels):
            if pixel_id % 1000 == 0:
                print(f"generating pixel {pixel_id}/{n_pixels}...")
            row = np.random.randint(0, 1500)
            col = np.random.randint(0, 1500)
            pixel_coordinates.append((row, col))
            tile_row = row // 375  #a 4x4 grid of tiles, this helps w/ spatial bias
            tile_col = col // 375
            tile_id = tile_row * 4 + tile_col #unique ID for each tile
            spatial_tiles.append(tile_id)

            t = np.arange(n_timesteps)

            #simulating seasonal patterns using harmonics - kinda like real satellite data
            seasonal_primary = 0.09 * np.sin(2 * np.pi * t / 12 + np.random.uniform(0, np.pi/2))
            seasonal_secondary = 0.05 * np.sin(4 * np.pi * t / 12 + np.random.uniform(0, np.pi/4))
            seasonal_tertiary = 0.03 * np.sin(6 * np.pi * t / 12 + np.random.uniform(0, np.pi/6))
            seasonal_quaternary = 0.02 * np.sin(8 * np.pi * t / 12 + np.random.uniform(0, np.pi/8)) #added this fourth one for fun
            seasonal_pattern = seasonal_primary + seasonal_secondary + seasonal_tertiary + seasonal_quaternary

            #Base index values with some noise
            base_ndvi = 0.75 + seasonal_pattern + np.random.normal(0, 0.025, n_timesteps) #typical forest NDVI
            base_nbr = 0.68 + 0.85 * seasonal_pattern + np.random.normal(0, 0.02, n_timesteps) #NBR usually tracks NDVI
            base_ndmi = 0.48 + 0.7 * seasonal_pattern + np.random.normal(0, 0.015, n_timesteps) #NDMI for water content
            base_evi = 0.52 + 0.9 * seasonal_pattern + np.random.normal(0, 0.018, n_timesteps) #EVI is another vegetation index

            #Deciding if a change happens and what kind
            #probabilities for different change types
            change_probabilities = [0.32, 0.22, 0.18, 0.16, 0.12] #probs: Stable, Minor, Moderate, Major, Regrowth
            change_type_idx = np.random.choice(5, p=change_probabilities)
            change_type = self.change_classes[change_type_idx]

            ndvi_series = base_ndvi.copy()
            nbr_series = base_nbr.copy()
            ndmi_series = base_ndmi.copy()
            evi_series = base_evi.copy()

            change_happened = False
            change_time = -1
            change_magnitude = 0
            change_persistence = 0

            if change_type != 'Stable Forest':
                change_happened = True
                #picking a random time for the change to happen, not too early or late
                change_time = np.random.randint(5, n_timesteps - 10) #avoiding edges

                #appling the change based on type
                if change_type == 'Minor Degradation':
                    mag_factor = np.random.uniform(-0.08, -0.15)
                    persist_factor = np.random.randint(3, 8)
                elif change_type == 'Moderate Deforestation':
                    mag_factor = np.random.uniform(-0.16, -0.3)
                    persist_factor = np.random.randint(6, 12)
                elif change_type == 'Major Deforestation':
                    mag_factor = np.random.uniform(-0.31, -0.6)
                    persist_factor = np.random.randint(10, 20)
                elif change_type == 'Fire Scar': # typo in comment
                    mag_factor = np.random.uniform(-0.5, -0.8) #big drop for fire
                    persist_factor = np.random.randint(8, 15)
                elif change_type == 'Regrowth/Recovery':
                    mag_factor = np.random.uniform(0.05, 0.2) #positive change for regrowth

                    #ensuring upper bound is greater than lower bound
                    upper_bound = n_timesteps - change_time
                    if upper_bound > 15:
                        persist_factor = np.random.randint(15, upper_bound) #can be long recovery
                    else:
                        persist_factor = 15 #Default to 15 if upper bound is not large enough


                change_magnitude = mag_factor * np.random.uniform(0.8, 1.2) #add some variability
                change_persistence = persist_factor

                #apply the change effect gradually over a few timesteps
                effect_duration = min(3, n_timesteps - change_time -1 ) #how long the drop/increase takes
                recovery_duration = min(change_persistence, n_timesteps - change_time - effect_duration -1) #how long it stays changed or recovers

                #initial drop/increase
                for i in range(effect_duration):
                    step_mag = (change_magnitude / effect_duration) * (i + 1)
                    ndvi_series[change_time + i + 1:] += step_mag
                    nbr_series[change_time + i + 1:] += step_mag * 0.9 #NBR slightly less sensitive
                    ndmi_series[change_time + i + 1:] += step_mag * 0.7 #NDMI even less
                    evi_series[change_time + i + 1:] += step_mag * 1.1 #EVI maybe more sensitive

                #maintain or recover
                if change_type in ['Minor Degradation', 'Moderate Deforestation', 'Major Deforestation', 'Fire Scar']:
                    #For degradation/deforestation, maintaining the low value with some noise
                    for i in range(recovery_duration):
                         ndvi_series[change_time + effect_duration + i + 1] += np.random.normal(0, 0.01) #small fluctuations
                         nbr_series[change_time + effect_duration + i + 1] += np.random.normal(0, 0.008)
                         ndmi_series[change_time + effect_duration + i + 1] += np.random.normal(0, 0.005)
                         evi_series[change_time + effect_duration + i + 1] += np.random.normal(0, 0.009)

                elif change_type == 'Regrowth/Recovery':
                    #for regrowth, simulating a recovery trend
                    recovery_amount = abs(change_magnitude) * np.random.uniform(0.5, 1.0) #recover part or all
                    for i in range(recovery_duration):
                        recovery_step = (recovery_amount / recovery_duration) * (i + 1)
                        ndvi_series[change_time + effect_duration + i + 1:] += recovery_step #gradual recovery
                        nbr_series[change_time + effect_duration + i + 1:] += recovery_step * 0.9
                        ndmi_series[change_time + effect_duration + i + 1:] += recovery_step * 0.75
                        evi_series[change_time + effect_duration + i + 1:] += recovery_step * 1.05

            #adding some general noise to the whole series(for good measure)
            ndvi_series += np.random.normal(0, 0.01, n_timesteps)
            nbr_series += np.random.normal(0, 0.008, n_timesteps)
            ndmi_series += np.random.normal(0, 0.005, n_timesteps)
            evi_series += np.random.normal(0, 0.009, n_timesteps)

            #make sure values are within a realistic range (0 to 1)
            ndvi_series = np.clip(ndvi_series, 0, 1)
            nbr_series = np.clip(nbr_series, 0, 1)
            ndmi_series = np.clip(ndmi_series, 0, 1)
            evi_series = np.clip(evi_series, 0, 1)


            time_series_ndvi.append(ndvi_series)
            time_series_nbr.append(nbr_series)
            time_series_ndmi.append(ndmi_series)
            time_series_evi.append(evi_series)
            labels.append(change_type)
            change_metadata.append({
                'pixel_id': pixel_id,
                'row': row, 'col': col,
                'change_happened': change_happened,
                'change_type': change_type,
                'change_time': change_time,
                'change_magnitude': change_magnitude,
                'change_persistence': change_persistence
            })

        self.time_series_ndvi = np.array(time_series_ndvi)
        self.time_series_nbr = np.array(time_series_nbr)
        self.time_series_ndmi = np.array(time_series_ndmi)
        self.time_series_evi = np.array(time_series_evi)
        self.labels = np.array(labels)
        self.pixel_coordinates = np.array(pixel_coordinates)
        self.change_metadata = change_metadata
        self.spatial_tiles = np.array(spatial_tiles)

        print(f"Generated {n_pixels} pixel time series.")
        print(f"time steps {n_timesteps}")
        print(f"change types {np.unique(labels)}")
        print(f"example labels {labels[:10]}") #showing a few examples
        print(f"spatial tiles generated {len(np.unique(spatial_tiles))}") #how many tiles?

        return self.time_series_ndvi, self.labels


    def perform_bfast_analysis(self):
        print("\nstage 2 Performing bfast analysis")

        change_candidates = []
        enhanced_ts_features = []
        n_pixels = len(self.time_series_ndvi)
        for pixel_idx in range(n_pixels):
            if pixel_idx % 1000 == 0:
                print(f"Processing pixel {pixel_idx}/{n_pixels}...")
            ndvi_ts = self.time_series_ndvi[pixel_idx]
            nbr_ts = self.time_series_nbr[pixel_idx]
            ndmi_ts = self.time_series_ndmi[pixel_idx]
            evi_ts = self.time_series_evi[pixel_idx]
            t_normalized = np.arange(len(ndvi_ts)) / 12.0
            X_design = np.column_stack([
                np.ones(len(ndvi_ts)),
                t_normalized,
                np.sin(2 * np.pi * t_normalized), np.cos(2 * np.pi * t_normalized),
                np.sin(4 * np.pi * t_normalized), np.cos(4 * np.pi * t_normalized),
                np.sin(6 * np.pi * t_normalized), np.cos(6 * np.pi * t_normalized),
                np.sin(8 * np.pi * t_normalized), np.cos(8 * np.pi * t_normalized)
            ])
            regression_model = HuberRegressor(epsilon=1.35, alpha=0.01, max_iter=200) if USE_HUBER else LinearRegression()
            regression_model.fit(X_design, ndvi_ts)
            predicted_ndvi = regression_model.predict(X_design)
            residuals = ndvi_ts - predicted_ndvi
            forest_density = np.median(ndvi_ts)
            if forest_density > 0.7:
                threshold_multiplier = 2.2 #high density forest
            elif forest_density > 0.5:
                threshold_multiplier = 2.8 #medium density forest
            elif forest_density > 0.3:
                threshold_multiplier = 3.5 #low density forest
            else:
                threshold_multiplier = 4.0 #sparse areas
            residual_std = np.std(residuals)
            change_threshold = threshold_multiplier * residual_std
            smoothed_residuals = (savgol_filter(residuals, window_length=5, polyorder=2)
                                  if USE_SAVGOL else gaussian_filter1d(residuals, sigma=1.0))
            peaks, _ = find_peaks(
                np.abs(smoothed_residuals),
                height=change_threshold,
                distance=4,
                prominence=change_threshold * 0.6,
                width=2
            )
            validated_changes = []
            for peak_idx in peaks:
                if 4 <= peak_idx <= len(ndvi_ts) - 4:
                    change_analysis = self._analyze_potential_change(
                        peak_idx, ndvi_ts, nbr_ts, ndmi_ts, evi_ts, residuals
                    )
                    if change_analysis['is_significant']:
                        validated_changes.append(change_analysis)
            if len(validated_changes) > 0:
                change_candidates.append(pixel_idx)
            ts_features = self._extract_time_series_features(
                ndvi_ts, nbr_ts, ndmi_ts, evi_ts, validated_changes,
                residuals, predicted_ndvi, regression_model.coef_
            )
            enhanced_ts_features.append(ts_features)
        self.enhanced_time_series_features = np.array(enhanced_ts_features)
        self.change_candidates = change_candidates
        print(f"bfast analysis complete. Found {len(change_candidates)} change candidates.")
        print(f"extracted {self.enhanced_time_series_features.shape[1]} time series feature(per pixel)")
        return change_candidates

    def _analyze_potential_change(self, change_idx, ndvi_ts, nbr_ts, ndmi_ts, evi_ts, residuals):
        n_obs = len(ndvi_ts)
        window_size = min(8, change_idx, n_obs - change_idx) #window size before and after
        before_start = max(0, change_idx - window_size)
        after_end = min(n_obs, change_idx + window_size)
        indices = {'ndvi': ndvi_ts, 'nbr': nbr_ts, 'ndmi': ndmi_ts, 'evi': evi_ts} if USE_MULTIINDEX else {'ndvi': ndvi_ts}
        index_changes = {}
        for index_name, index_ts in indices.items():
            pre_values = index_ts[before_start:change_idx]
            post_values = index_ts[change_idx:after_end]
            if len(pre_values) > 2 and len(post_values) > 2:
                magnitude = np.mean(post_values) - np.mean(pre_values) #bascally how big was the change?
                t_stat, p_value = stats.ttest_ind(pre_values, post_values, equal_var=False) #doing a t-test
                index_changes[index_name] = {
                    'magnitude': magnitude,
                    'p_value': p_value,
                    'significant': p_value < 0.05 and abs(magnitude) > 0.03 #statistically significant?
                }
            else:
                index_changes[index_name] = {
                    'magnitude': 0, 'p_value': 1.0, 'significant': False # not enuough data for test
                }
        ndvi_change = index_changes['ndvi']
        validation_score = 1.0                                                                       #base score
        if USE_MULTIINDEX:
            for other_index in ['nbr', 'ndmi', 'evi']:
                if other_index in index_changes:
                    other_change = index_changes[other_index]
                    same_direction = np.sign(ndvi_change['magnitude']) == np.sign(other_change['magnitude']) #change in same direction?
                    both_significant = ndvi_change['significant'] and other_change['significant']             #both changes significant?
                    if same_direction and both_significant:
                        validation_score += 0.4 #good sign
                    elif same_direction:
                        validation_score += 0.2 #okay sign
                    else:
                        validation_score -= 0.3 #bad sign, maybe not real change
        primary_magnitude = ndvi_change['magnitude']
        abs_magnitude = abs(primary_magnitude)
        if primary_magnitude < 0:                                      #if NDVI dropped
            if abs_magnitude > self.forest_change_thresholds['fire_scar']:
                change_type = 'fire_scar'
            elif abs_magnitude > self.forest_change_thresholds['major_deforestation']:
                change_type = 'major_deforestation'
            elif abs_magnitude > self.forest_change_thresholds['moderate_deforestation']:
                change_type = 'moderate_deforestation'
            elif abs_magnitude > self.forest_change_thresholds['minor_degradation']:
                change_type = 'minor_degradation'
            else:
                change_type = 'seasonal_variation' #just seasonal?
        else:                                                        #if NDVI increased
            if abs_magnitude > abs(self.forest_change_thresholds['regrowth']):
                change_type = 'regrowth' #recovery or growth?
            else:
                change_type = 'seasonal_variation' #probably seasonal
        persistence = self._calculate_change_persistence(change_idx, ndvi_ts, primary_magnitude) #how long did it last?
        is_significant = (                                                                        #putting it all together
            validation_score > 1.0 and
            abs_magnitude > 0.05 and                                                 #needs to be noticeable
            persistence >= 2 and                                                    #needs to last more than one timestep(obviously)
            change_type != 'seasonal_variation' #not just seasonal
        )
        return {
            'index': change_idx,
            'magnitude': primary_magnitude,
            'change_type': change_type,
            'validation_score': min(validation_score, 4.0), #cap score
            'persistence': persistence,
            'is_significant': is_significant,
            'confidence': 1 - ndvi_change['p_value'], #confidence from p-value
            'multi_index_changes': index_changes
        }

    def _calculate_change_persistence(self, change_idx, ndvi_ts, magnitude):
        n_obs = len(ndvi_ts)
        persistence = 1
        threshold = abs(magnitude) * 0.4 #how much change needs to be maintained
        baseline_value = ndvi_ts[change_idx - 1] if change_idx > 0 else ndvi_ts[0] #value before change
        for i in range(change_idx + 1, min(change_idx + 15, n_obs)):
            current_change = abs(ndvi_ts[i] - baseline_value)
            if current_change > threshold:
                persistence += 1
            else:
                break #didnt persist
        return persistence

    def _extract_time_series_features(self, ndvi_ts, nbr_ts, ndmi_ts, evi_ts, validated_changes, residuals, predicted, coeffs):
        features = []
        for index_ts in [ndvi_ts, nbr_ts, ndmi_ts, evi_ts]: #eatures for each index time series
            features.extend([np.mean(index_ts), np.std(index_ts), np.min(index_ts), np.max(index_ts)])
        trend_slope = coeffs[1]                                                                              #slope from regression
        trend_curvature = np.polyfit(range(len(ndvi_ts)), ndvi_ts, 2)[0] #how trend bends
        features.extend([trend_slope, abs(trend_slope), trend_curvature])
        seasonal_1_amplitude = np.sqrt(coeffs[2]**2 + coeffs[3]**2)                                  #amplitude of 1st harmonic
        seasonal_2_amplitude = np.sqrt(coeffs[4]**2 + coeffs[5]**2)                                #amplitude of 2nd harmonic
        seasonal_3_amplitude = np.sqrt(coeffs[6]**2 + coeffs[7]**2)                              #amplitude of 3rd harmonic
        seasonal_4_amplitude = np.sqrt(coeffs[8]**2 + coeffs[9]**2)                            #amplitude of 4th harmonic
        total_seasonal_amplitude = seasonal_1_amplitude + seasonal_2_amplitude + seasonal_3_amplitude + seasonal_4_amplitude           #sum
        features.extend([seasonal_1_amplitude, seasonal_2_amplitude, seasonal_3_amplitude, seasonal_4_amplitude, total_seasonal_amplitude])
        forest_density_score = np.mean(ndvi_ts > self.min_forest_ndvi)                                                           #proportion of timesteps as forest
        vegetation_vigor = np.mean(ndvi_ts[ndvi_ts > 0.3]) if np.any(ndvi_ts > 0.3) else 0                                    #avg ndvi during growing season
        features.extend([forest_density_score, vegetation_vigor])
        n_changes = len(validated_changes)                                                                                #number of significant changes found
        change_frequency = n_changes / len(ndvi_ts)                                                                   #how often changes happen
        features.extend([n_changes, change_frequency])
        if validated_changes:                                                                                                #features related to changes found
            magnitudes = [change['magnitude'] for change in validated_changes]
            max_change_magnitude = max(magnitudes, key=abs)                                                          #biggest change
            mean_validation_score = np.mean([change['validation_score'] for change in validated_changes]) #avg validation score
            mean_persistence = np.mean([change['persistence'] for change in validated_changes])              #avg persistence
            change_diversity = len(set([change['change_type'] for change in validated_changes]))    #types of changes found
            features.extend([max_change_magnitude, mean_validation_score, mean_persistence, change_diversity])
        else:
            features.extend([0, 0, 0, 0])
        features.extend([
            np.std(residuals), np.max(np.abs(residuals)),
            stats.skew(residuals), stats.kurtosis(residuals)
        ])
        return features

    def extract_cnn_spatial_features(self):
        print("\nStage 3 Extracting cnn Spatial Features (placeholder)")

        print("here I am simulating CNN features for demo.")
        print("in an actual use case this would most likely be replace with real cnn embeddings.")
        n_cnn_features = 512
        cnn_features = []
        n_pixels = len(self.time_series_ndvi)
        for pixel_idx in range(n_pixels):
            if pixel_idx % 1000 == 0:
                print(f"Generating features for pixel {pixel_idx}/{n_pixels}...")
            row, col = self.pixel_coordinates[pixel_idx]
            ndvi_ts = self.time_series_ndvi[pixel_idx]
            spatial_features = np.random.normal(0, 1, n_cnn_features) #random features
            ts_variance = np.var(ndvi_ts)                             #variance over t
            ts_trend = np.polyfit(range(len(ndvi_ts)), ndvi_ts, 1)[0] #trend over t
            ts_mean = np.mean(ndvi_ts)                                 #average ndvi
            ts_range = np.max(ndvi_ts) - np.min(ndvi_ts)               #rang of ndvi
            spatial_features[:100] += ts_variance * 5 + np.random.normal(0, 0.1, 100) #injecting some time series info
            spatial_features[100:200] += ts_trend * 10 + np.random.normal(0, 0.1, 100)
            spatial_features[200:300] += (ts_mean - 0.5) * 3 + np.random.normal(0, 0.1, 100)
            spatial_features[300:400] += ts_range * 4 + np.random.normal(0, 0.1, 100)
            spatial_features[400:450] += 0.3 * np.sin(row / 50) + 0.2 * np.cos(col / 50) #inject spatial info
            spatial_features[450:500] += 0.2 * (row % 100) / 100 + 0.2 * (col % 100) / 100
            center_distance = np.sqrt((row - 750)**2 + (col - 750)**2)                    #dist from center
            edge_proximity = min(row, col, 1500-row, 1500-col) / 1500                     #dist to nearest edge
            spatial_features[500:512] += (0.15 * np.sin(center_distance / 100) +
                                          0.1 * edge_proximity +
                                          np.random.normal(0, 0.05, 12))                  #inject more spatial info
            cnn_features.append(spatial_features)
        self.cnn_spatial_features = np.array(cnn_features)
        print(f"CNN feature simulation complete.")
        print(f"Extracted {n_cnn_features} features per pixel.")
        return self.cnn_spatial_features

    def extract_ancillary_features(self):
        print("\nStage 4: Extracting Ancillary Features")

        ancillary_features = []
        for pixel_idx in range(len(self.time_series_ndvi)):
            row, col = self.pixel_coordinates[pixel_idx]
            features = []
            features.extend([row / 1500, col / 1500, (row * col) / (1500 * 1500)])                 #normalized coords
            center_distance = np.sqrt((row - 750)**2 + (col - 750)**2) / 1061
            edge_distance = min(row, col, 1500-row, 1500-col) / 750                                #normalized dist to edge
            features.extend([center_distance, edge_distance])
            features.extend([
                np.sin(row / 100), np.cos(row / 100),
                np.sin(col / 100), np.cos(col / 100),
                np.sin(2 * np.pi * row / 200), np.cos(2 * np.pi * col / 200)
            ])
            ndvi_ts = self.time_series_ndvi[pixel_idx]
            nbr_ts = self.time_series_nbr[pixel_idx]
            ndmi_ts = self.time_series_ndmi[pixel_idx]
            evi_ts = self.time_series_evi[pixel_idx]
            try:                                               #corrs between indices
                ndvi_nbr_corr = np.corrcoef(ndvi_ts, nbr_ts)[0, 1]
                ndvi_ndmi_corr = np.corrcoef(ndvi_ts, ndmi_ts)[0, 1]
                ndvi_evi_corr = np.corrcoef(ndvi_ts, evi_ts)[0, 1]
                ndvi_nbr_corr = 0 if np.isnan(ndvi_nbr_corr) else ndvi_nbr_corr
                ndvi_ndmi_corr = 0 if np.isnan(ndvi_ndmi_corr) else ndmi_ndmi_corr
                ndvi_evi_corr = 0 if np.isnan(ndvi_evi_corr) else ndvi_evi_corr
            except:                                                #handle cases where correlation cant be calc
                ndvi_nbr_corr = ndmi_ndmi_corr = ndvi_evi_corr = 0
            features.extend([ndvi_nbr_corr, ndvi_ndmi_corr, ndvi_evi_corr])
            mean_ndvi = np.mean(ndvi_ts) #avg values of indices
            mean_nbr = np.mean(nbr_ts)
            mean_ndmi = np.mean(ndmi_ts)
            mean_evi = np.mean(evi_ts)
            features.extend([                                      #ratios and diffs between indices
                mean_ndvi / (mean_nbr + 0.001),
                mean_ndvi - mean_ndmi,
                (mean_ndvi + mean_evi) / 2,
                np.std([mean_ndvi, mean_nbr, mean_ndmi, mean_evi]) #std dev of mean values
            ])
            ancillary_features.append(features)
        self.ancillary_features = np.array(ancillary_features)
        print(f"ancillary feature extraction done.")
        print(f"extracted {self.ancillary_features.shape[1]} features(per pixel)")
        return self.ancillary_features

    def create_spatial_train_test_split(self, test_size=0.25, random_state=42):
        print("\ncreating spatial train/test split...")
        print(f"using a random state {random_state}")
        unique_tiles = np.unique(self.spatial_tiles)
        n_tiles = len(unique_tiles)
        print(f"total tiles available {n_tiles}")
        n_test_tiles = max(1, int(n_tiles * test_size))
        n_train_tiles = n_tiles - n_test_tiles
        tile_random = np.random.RandomState(random_state)
        tile_random.shuffle(unique_tiles)
        test_tiles = unique_tiles[:n_test_tiles]
        train_tiles = unique_tiles[n_test_tiles:]
        train_mask = np.isin(self.spatial_tiles, train_tiles)
        test_mask = np.isin(self.spatial_tiles, test_tiles)
        print(f"gave {n_train_tiles} tiles for training and {n_test_tiles} for testing.")
        print(f"pixel counts {np.sum(train_mask)} training pixels, {np.sum(test_mask)} test pixels.")
        print(f"test split ratio {np.sum(test_mask)/(np.sum(train_mask)+np.sum(test_mask)):.3f}")
        return train_mask, test_mask

    def train_classifier_model(self, test_size=0.25):
        print("\nstage 5 Training gb classifier")

        all_features = np.concatenate([
            self.cnn_spatial_features,
            self.enhanced_time_series_features,
            self.ancillary_features
        ], axis=1)

        #just to make sure everything is working fine till now
        print(f"otal features for training {all_features.shape[1]}")
        print(f"cnn spatial features {self.cnn_spatial_features.shape[1]}")
        print(f"time series features {self.enhanced_time_series_features.shape[1]}")
        print(f"ancillary features {self.ancillary_features.shape[1]}")

        y_encoded = self.label_encoder.fit_transform(self.labels)
        train_mask, test_mask = self.create_spatial_train_test_split(test_size, random_state=42)
        X_train = all_features[train_mask]
        X_test = all_features[test_mask]
        y_train = y_encoded[train_mask]
        y_test = y_encoded[test_mask]
        X_train_scaled = self.feature_scaler.fit_transform(X_train)
        X_test_scaled = self.feature_scaler.transform(X_test)
        print("\nperforming a feature selection...")
        self.feature_selector = SelectKBest(score_func=f_classif, k=min(200, all_features.shape[1]))
        X_train_selected = self.feature_selector.fit_transform(X_train_scaled, y_train)
        X_test_selected = self.feature_selector.transform(X_test_scaled)
        print(f"selected {X_train_selected.shape[1]} features.")
        base_gbm = GradientBoostingClassifier(
            n_estimators=300,
            max_depth=8,
            learning_rate=0.08,
            subsample=0.85,
            max_features='sqrt',
            random_state=42,
            validation_fraction=0.15,
            n_iter_no_change=20
        )
        print("training the gb model...")
        if USE_CALIBRATION:
            print("applying probability calibr.")
            self.gbm_model = CalibratedClassifierCV(base_gbm, method='sigmoid', cv=3)
        else:
            self.gbm_model = base_gbm
        self.gbm_model.fit(X_train_selected, y_train)
        y_pred = self.gbm_model.predict(X_test_selected)
        y_pred_proba = self.gbm_model.predict_proba(X_test_selected)
        accuracy = accuracy_score(y_test, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
        print(f"\ntraining done")
        print(f"accuracy {accuracy:.4f} ({accuracy*100:.1f}%)")
        print(f"Weighted F1-Score: {f1:.4f}")
        if USE_CALIBRATION:
            print(f"calibration applied.")
        self.model_metrics = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'n_features_selected': X_train_selected.shape[1],
            'n_training_samples': X_train.shape[0],
            'calibrated': USE_CALIBRATION
        }
        class_names = self.label_encoder.classes_
        print(f"\nClassification Report")
        print(classification_report(y_test, y_pred, target_names=class_names))
        self._create_confusion_matrix_plot(y_test, y_pred, class_names)
        return X_test_selected, y_test, y_pred, class_names, y_pred_proba

    def _create_confusion_matrix_plot(self, y_true, y_pred, class_names):
        from sklearn.metrics import confusion_matrix
        print("\ncreating confusion matrix plot")
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(10, 8))
        plt.imshow(cm, interpolation='nearest', cmap='Blues')
        plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45, ha='right')
        plt.yticks(tick_marks, class_names)
        thresh = cm.max() / 2.
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                plt.text(j, i, format(cm[i, j], 'd'),
                        horizontalalignment="center",
                        color="white" if cm[i, j] > thresh else "black",
                        fontsize=12, fontweight='bold')
        plt.ylabel('True Label', fontsize=14, fontweight='bold')
        plt.xlabel('Predicted Label', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('forest_confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("Confusion matrix plot saved as 'forest_confusion_matrix.png'.")
        per_class_f1 = precision_recall_fscore_support(y_true, y_pred, average=None)[2]
        print(f"\nper Class f1 Scores")
        for i, (class_name, f1_score) in enumerate(zip(class_names, per_class_f1)):
            print(f"{class_name}: {f1_score:.3f}")

In [ ]:
def run_complete_system():
    print("executing the complete system")

    system = ForestChangeDetectionSystem("amazon forest")
    print("Starting the process...")
    time_series_data, labels = system.simulate_satellite_data(n_pixels=7500, n_timesteps=48)
    change_candidates = system.perform_bfast_analysis()
    cnn_features = system.extract_cnn_spatial_features()
    ancillary_features = system.extract_ancillary_features()
    test_results = system.train_classifier_model()
    X_test, y_test, y_pred, class_names, y_pred_proba = test_results


    print("\nexecution complete")

    print(f"\nFinal performance summary:")
    print(f"test Accuracy {system.model_metrics['accuracy']:.1%}")
    print(f"weighted f1 Score {system.model_metrics['f1_score']:.4f}")
    print(f"number of features used {system.model_metrics['n_features_selected']}")
    print(f"change rate detected {len(system.change_candidates)/len(system.time_series_ndvi)*100:.1f}%")
    print(f"probability calibration Applied {'Yes' if system.model_metrics['calibrated'] else 'No'}")

    with open('final_forest_change_system.pkl', 'wb') as f:
        pickle.dump({
            'system': system,
            'metrics': system.model_metrics,
            'test_results': (X_test, y_test, y_pred, class_names, y_pred_proba),
            'ablation_config': {
                'USE_HUBER': USE_HUBER,
                'USE_MULTIINDEX': USE_MULTIINDEX,
                'USE_SAVGOL': USE_SAVGOL,
                'USE_CALIBRATION': USE_CALIBRATION
            }
        }, f)
    print(f"\nSystem object saved to 'final_forest_change_system.pkl'.")
    print(f"Confusion matrix plot saved to 'forest_confusion_matrix.png'.")
    return system

In [ ]:
print("starting final system execution...")
system = run_complete_system()
print("Whole thing is complete")
